# Compile-only Results

In [1]:
import db as db
import sqlalchemy as sql
from db import *
%run util.py

hostname = "cc1"
db.init(hostname)

'mysql://cc1:3306/project_b'

In [2]:
TABLE_NAME = "CLgen"
TABLE = coCLgenResult

In [3]:
from labm8 import crypto
print(crypto.md5_str(TABLE.__name__))

11ed7e305bfa90d6f6bcd31fc9c98e15


In [4]:
import cldrive
import sys

def get_co_report(result_id: int, table=coCLgenResult, report_type: str="bf"):
    with Session(commit=False) as s:
        result = s.query(table).filter(table.id == result_id).first()

        if not result:
            raise KeyError(f"no result with ID {result_id}")

        flags = result.params.to_flags()
        program = result.program

        # generate bug report
        now = datetime.datetime.utcnow().isoformat()

        bug_type = {
            "bf": "compilation failure",
            "c": "runtime crash"
        }[report_type]

        report_id = crypto.md5_str(table.__name__) + "-" + str(result.id)
        result_output = '\n'.join([f'//   {line}' for line in result.stderr.strip().split('\n')
                                   if not line.startswith("+ ")])
        header = f"""
// bug report {report_id}.c
// 
// Metadata:
//   OpenCL platform:        {result.testbed.platform}
//   OpenCL device:          {result.testbed.device}
//   Driver version:         {result.testbed.driver}
//   OpenCL version:         {result.testbed.opencl}
//   Host operating system:  {result.testbed.host}
//   OpenCL optimizations:   {result.params.optimizations_on_off}
//
// Output:
{result_output}
//   [Return code {result.status}]
//\
"""
        src = cldrive.emit_c(
            env=cldrive.make_env(),
            src=result.program.src,
            compile_only=True,
            create_kernel=result.params.build_kernel,
            gsize=None, lsize=None, inputs=None,
        )
        return (header + src).strip()

Segfaults:

In [5]:
from collections import defaultdict
from labm8 import fs

outbox = fs.path("outbox", "coCLgenResults", "segfaults")
fs.mkdir(outbox)
with Session(commit=True) as s:
    q = s.query(TABLE)\
        .filter(TABLE.status == 139,
                sql.or_(TABLE.submitted == None, TABLE.submitted == 0),
                sql.or_(TABLE.dupe == None, TABLE.dupe == 0))\
        .order_by(TABLE.id)

    dupes = 0
    errs = set()
    for result in q:
        key = result.testbed_id, result.program_id

        if key in errs:
            dupes += 1
            result.dupe = 1
            continue
        errs.add(key)

        vendor = PLATFORMS_2_VENDORS[result.testbed.platform]
        outpath = fs.path(outbox, f"bug-report-{vendor}-{result.id}.c")            

        report = get_co_report(**{
            "result_id": result.id,
            "table": coCLgenResult,
        })

        with open(outpath, "w") as outfile:
            print(report, file=outfile)
        result.submitted = 1
        print(outpath)
print("done")
print(f"{dupes} duplicates flagged")

done
0 duplicates flagged


In [14]:
with Session(commit=True) as s:
    q = s.query(TABLE)\
        .filter(TABLE.classification == "Build failure",
                sql.or_(TABLE.submitted == None, TABLE.submitted == 0),
                sql.or_(TABLE.dupe == None, TABLE.dupe == 0))\
        .order_by(TABLE.id)

    print(q.count())
    for result in q:
        print(result.stderr, '\n')

0
